In [1]:
import torch
print(torch.__version__)

2.9.0+cu126


In [ ]:
!pip install autoawq

In [ ]:
!pip install transformers==4.51.3

In [4]:
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import statistics

model_id = "hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4"

device = "cuda"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
)

prompt = "What is LLM?"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# --- warmup ---
_ = model.generate(**inputs, max_new_tokens=1)
torch.cuda.synchronize()

# --- repeated TTFT measurement ---
iters = 20
latencies = []

for _ in range(iters):
    torch.cuda.synchronize()
    start = time.time()

    _ = model.generate(**inputs, max_new_tokens=1)

    torch.cuda.synchronize()
    end = time.time()

    latencies.append((end - start) * 1000)  # ms

# --- statistics ---
mean = statistics.mean(latencies)
median = statistics.median(latencies)

print(f"TTFT mean   : {mean:.2f} ms")
print(f"TTFT median : {median:.2f} ms")
print(f"Raw first 5 latencies: {[round(x,2) for x in latencies[:5]]}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id`

TTFT mean  : 190.99 ms
TTFT p50   : 190.81 ms
Raw first 5 latencies: [191.82, 196.6, 190.47, 188.26, 192.44]
